# Embryo Data Query

This notebook queries embryo-related data from multiple databases for a given prontuario/PIN.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_congelamentos_embrioes
  - view_descongelamentos_embrioes
  - view_embrioes_congelados
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - gold.clinisys_embrioes
  - gold.embryoscope_embrioes
  - gold.embryoscope_clinisys_combined
  - gold.embryoscope_treatment (from embryoscope_embrioes)
  - silver.planilha_embriologia
  - gold.planilha_embryoscope_combined


In [79]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connections


In [80]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Query Functions - Clinisys Tables


In [81]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [82]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    return df

def query_descongelamentos_embrioes(conn, prontuario):
    """Query view_descongelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_embrioes")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    return df

def query_descongelamentos_ovulos(conn, prontuario):
    """Query view_descongelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_ovulos")
    return df

def query_ovulos_congelados(conn, prontuario):
    """Query view_ovulos_congelados for a specific prontuario"""
    print(f"\n=== VIEW_OVULOS_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_ovulos_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_ovulos_congelados")
    return df


## Query Functions - Huntington Data Lake Tables


In [83]:
def query_clinisys_embrioes(conn, prontuario):
    """Query gold.clinisys_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.CLINISYS_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.clinisys_embrioes 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.clinisys_embrioes")
    return df

def query_embryoscope_embrioes(conn, prontuario):
    """Query gold.embryoscope_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY embryo_KIDDate DESC, treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_embrioes")
    return df

def query_embryoscope_clinisys_combined(conn, prontuario):
    """Query gold.embryoscope_clinisys_combined for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===")
    query = f"""
    SELECT * FROM gold.embryoscope_clinisys_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, micro_codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_clinisys_combined")
    return df


In [84]:
def query_planilha_embriologia(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia 
    WHERE PIN = {prontuario}
    ORDER BY "DATA DA FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia")
    return df

def query_planilha_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a specific prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT * FROM gold.planilha_embryoscope_combined 
    WHERE "micro_prontuario" = {prontuario} OR "planilha_PIN" = {prontuario}
    ORDER BY "descong_em_DataTransferencia" DESC, "planilha_DATA_DA_FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.planilha_embryoscope_combined")
    return df


## Main Query Function


In [85]:
def query_embryo_data(prontuario):
    """
    Query all embryo-related data for a given prontuario/PIN.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING EMBRYO DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_descongelamentos_embrioes = query_descongelamentos_embrioes(clinisys_con, prontuario)
        df_embrioes_congelados = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_descongelamentos_ovulos = query_descongelamentos_ovulos(clinisys_con, prontuario)
        df_ovulos_congelados = query_ovulos_congelados(clinisys_con, prontuario)
        
        # Query huntington data lake tables
        df_clinisys_embrioes = query_clinisys_embrioes(huntington_con, prontuario)
        df_embryoscope_embrioes = query_embryoscope_embrioes(huntington_con, prontuario)
        df_embryoscope_clinisys_combined = query_embryoscope_clinisys_combined(huntington_con, prontuario)
        # df_embryoscope_treatment = query_embryoscope_treatment(huntington_con, prontuario)
        df_planilha_embriologia = query_planilha_embriologia(huntington_con, prontuario)
        df_planilha_embryoscope_combined = query_planilha_embryoscope_combined(huntington_con, prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_descongelamentos_embrioes: {len(df_descongelamentos_embrioes)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes_congelados)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_descongelamentos_ovulos: {len(df_descongelamentos_ovulos)} records")
        print(f"- view_ovulos_congelados: {len(df_ovulos_congelados)} records")
        print(f"- gold.clinisys_embrioes: {len(df_clinisys_embrioes)} records")
        print(f"- gold.embryoscope_embrioes: {len(df_embryoscope_embrioes)} records")
        print(f"- gold.embryoscope_clinisys_combined: {len(df_embryoscope_clinisys_combined)} records")
        print(f"- silver.planilha_embriologia: {len(df_planilha_embriologia)} records")
        print(f"- gold.planilha_embryoscope_combined: {len(df_planilha_embryoscope_combined)} records")
        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'descongelamentos_embrioes': df_descongelamentos_embrioes,
            'embrioes_congelados': df_embrioes_congelados,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'descongelamentos_ovulos': df_descongelamentos_ovulos,
            'ovulos_congelados': df_ovulos_congelados,
            'clinisys_embrioes': df_clinisys_embrioes,
            'embryoscope_embrioes': df_embryoscope_embrioes,
            'embryoscope_clinisys_combined': df_embryoscope_clinisys_combined,
            'planilha_embriologia': df_planilha_embriologia,
            'planilha_embryoscope_combined': df_planilha_embryoscope_combined
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Example Usage


In [220]:
# Example usage
prontuario = 156766
# Query all embryo data
results = query_embryo_data(prontuario)



QUERYING EMBRYO DATA FOR PRONTUARIO: 156766
Query executed at: 2026-01-13 17:32:18.386895

=== VIEW_MICROMANIPULACAO ===
Found 5 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 19 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 5 records in view_tratamentos

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 4 records in view_congelamentos_embrioes

=== VIEW_DESCONGELAMENTOS_EMBRIOES ===
Found 0 records in view_descongelamentos_embrioes

=== VIEW_EMBRIOES_CONGELADOS ===
Found 4 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 1 records in view_congelamentos_ovulos

=== VIEW_DESCONGELAMENTOS_OVULOS ===
Found 1 records in view_descongelamentos_ovulos

=== VIEW_OVULOS_CONGELADOS ===
Found 8 records in view_ovulos_congelados

=== GOLD.CLINISYS_EMBRIOES ===
Found 19 records in gold.clinisys_embrioes

=== GOLD.EMBRYOSCOPE_EMBRIOES ===
Found 15 records in gold.embryoscope_embrioes

=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINE

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Found 19 records in gold.planilha_embryoscope_combined

QUERY SUMMARY FOR PRONTUARIO 156766
- view_micromanipulacao: 5 records
- view_micromanipulacao_oocitos: 19 records
- view_tratamentos: 5 records
- view_congelamentos_embrioes: 4 records
- view_descongelamentos_embrioes: 0 records
- view_embrioes_congelados: 4 records
- view_congelamentos_ovulos: 1 records
- view_descongelamentos_ovulos: 1 records
- view_ovulos_congelados: 8 records
- gold.clinisys_embrioes: 19 records
- gold.embryoscope_embrioes: 15 records
- gold.embryoscope_clinisys_combined: 19 records
- silver.planilha_embriologia: 0 records
- gold.planilha_embryoscope_combined: 19 records

Database connections closed.


## Display Results

### 1. Micromanipulacao


In [221]:
print("=== MICROMANIPULACAO DATA ===")
micromanipulacao_data = results['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulacao data found.")


=== MICROMANIPULACAO DATA ===
Records: 5


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,recepcao_ovulos,Denud_DL,checagem_fertilizacao,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,ICSIDescongelados,pgd_oocito,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,unidade_id,responsavel_denud_id,profissional_fertilizacao_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,materiais_insumos,local_coleta,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,observacoes_resumo_ciclos,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,conc_ps1,EstadoSptz,check_gradiente_ps,check_swimup,volume_ps2,concentracao_progressivos_pos,conc_ps2,comentarios_ps,responsavel_ps,timelapse,transferidos,Transfer_D5,medico_responsavel_id,azoospermia,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,26281,2218/25,156766,44,42,2025-07-02,15:30,15:40,12:20,10,3492,Não,15:20,08:00,08:00,08:00,08:00,None,3,3,Não,1,None,2025-07-02,ICSI,02 espermatozoides móveis progressivos\r\n,02 espermatozoides normais\r\n,None,0000000001,3548,3492,3492,4005,3492,None,None,Laboratório,Normal,"0,85",02,03,1,None,None,None,None,None,None,None,None,01,None,None,Ejaculado,13:11,13:54,"3,5",Não,"20,0","26,0",Fresco,None,None,"0,2","3,0","3,0",None,4003,Sim,0,Sim,10,Nenhum,None,None,None,esposa,2,2,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,IB1357/25,Sim,fd23f973edc4b5469d6537be48c3823c,2025-07-21 21:39:47
1,24791,873/25,156766,44,41,2025-03-23,13:30,13:40,11:50,10,3613,Não,13:15,08:00,08:00,08:00,None,None,1,1,Não,None,1,2025-03-23,ICSI,01 espermatozoide móvel progressivo\r\n,01 espermatozoide normal\r\n,None,0000000001,3613,3652,3755,3505,None,None,None,Laboratório,Normal,"0,85",02,03,1,None,None,None,None,None,None,None,ES 2,01,None,None,Ejaculado,12:21,13:09,"2,0",Não,"6,0","15,0",Fresco,None,None,"0,2","1,0","1,0",None,4003,Sim,0,Sim,10,Nenhum,None,None,None,esposa,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,IB537/25,Sim,4dd34aafa80d97492dfa73afe72cc371,2025-07-21 21:39:47
2,22794,3524/24,156766,43,41,2024-10-19,11:45,11:55,09:20,3,3613,Não,11:40,08:00,08:00,08:00,08:00,None,7,3,Não,None,1,2024-10-19,ICSI,03 espermatozoides móveis progressivos\r\n,03 espermatozoides normais\r\n,1,0000000001,4005,5997,3505,4005,5997,None,None,Laboratório,Normal,"1,5",02,03,None,None,None,None,None,None,None,None,ES 2,02,None,None,Ejaculado,10:05,11:08,"2,0",Não,"34,0","58,0",Fresco,None,1,"0,5","20,0","20,0",Paciente relatou perda de material da porção final do ejaculado.,7306,Sim,0,Sim,3,Nenhum,None,None,None,esposa,3,3,0,0,3,1,0,2,1,0,0,0,0,0,0,0,0,IB1700/24,Sim,18e71da5e316ac54f4e7eb6065375162,2025-07-21 21:39:47
3,19589,626/24,156766,43,40,2024-03-01,14:10,14:35,None,3,None,Não,None,08:00,08:00,08:00,08:00,08:00,None,None,Sim,1,1,2024-03-01,ICSI,08 espermatozoides móveis progressivos\r\n,08 espermatozoides normais\r\n,1,0000000001,None,3548,3505,3613,3512,5997,None,Laboratório,Normal,"1,5",02,03,None,None,None,None,None,None,None,None,ES 1,04,None,None,Ejaculado,08:26,09:26,"3,1",None,"31,0","41,0",Fresco,No

### 2. Micromanipulacao Oocitos


In [222]:
print("=== MICROMANIPULACAO OOCITOS DATA ===")
oocitos_data = results['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulacao oocitos data found.")


=== MICROMANIPULACAO OOCITOS DATA ===
Records: 19


,id,id_micromanipulacao,diaseguinte,Maturidade,Embriologista,PI,TCD,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,OocitoDoado,OrigemOocito,InseminacaoOocito,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,175967,18569,Não,MII,4005,None,Descartado,None,None,None,DEG,C1,Fresco,Não,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1472ad528d81965dca8294dbec19261b,2025-07-21 21:40:02,1,1
1,175968,18569,Não,MII,4005,None,Descartado,None,None,None,DEG,C1,Fresco,Não,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,b06e020a03d1a60600ec9f5b1cf3e942,2025-07-21 21:40:02,1,2
2,175969,18569,Não,MII,4005,None,Descartado,None,None,None,2PN,C1,Fresco,Não,Fresco,ICSI,Sim,10,1,Simétricos,None,None,None,None,None,Sim,16,Fragmentação 1,None,None,Sim,None,Mórula C,None,None,Não,None,None,None,None,None,None,8d5dd4ef15cf5d66615a3adccfd9c5d2,2025-07-21 21:40:02,1,3
3,175970,18569,Não,Alterado,None,None,Descartado,None,None,None,None,C0,None,Não,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,fa26d7df8367b850b2487acb375a859f,2025-07-21 21:40:02,0,4
4,186310,19589,Não,MII,3505,5997,Criopreservado,Sim,Aneuploide,PGT-A,2PN,None,Fresco,Não,Descongelado,ICSI,Sim,8,1,Simétricos,None,None,None,None,None,Sim,None,Blastocisto Grau 1,None,None,Sim,None,Blastocisto Grau 5,B,B,None,None,None,None,None,None,None,bd428992dbd03ae1f1204f1fe39f88c0,2025-07-21 21:40:02,1,1
5,186311,19589,Não,MII,3505,None,Descartado,None,None,None,1PN,None,Fresco,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,09df5d2b890119599933fbcbd3a25198,2025-07-21 21:40:02,1,2
6,186312,19589,Não,MII,3505,None,Descartado,None,None,None,1CP,None,Fresco,Não,Descongelado,ICSI,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,56d1d2b62b399a48d728e3ed4776661f,2025-07-21 21:40:02,1,3
7,186313,19589,Não,MII,3505,None,Descartado,None,None,None,3PN+,None,Fresco,Não,Descongelado,ICSI,Sim,6,2,Simétricos,None,None,None,None,None,Sim,None,Mórula C,None,None,Sim,None,Blastocisto Grau 1,None,None,Não,None,None,None,None,None,None,a9d5088afe72eada8160fbbb0cc86fed,2025-07-21 21:40:02,1,4
8,186314,19589,Não,MII,3505,None,Descartado,None,None,None,3PN+,None,Fresco,Não,Descongelado,ICSI,Sim,6,3,Simétricos,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,90b4a3edbdbc57ee730246ac2bfff499,2025-07-21 21:40:02,1,5
9,186315,19589,Não,MII,3505,None,Descartado,None,None,None,2PN,None,Fresco,Não,Descongelado,ICSI,Sim,5,2,Simétricos,None,None,None,None,None,Sim,11,Fragmentação 2,None,None,Sim,None,Blastocisto Grau 2,None,None,Não,None,None,None,None,None,None,5a17393c8eda07b2dcf40aadf27bc8d3,2025-07-21 21:40:02,1,6


### 3. Tratamentos


In [223]:
print("=== TRATAMENTOS DATA ===")
tratamentos_data = results['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No tratamentos data found.")


=== TRATAMENTOS DATA ===
Records: 5


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,tipo_procedimento,tipo_inducao,protocolo,tipo_preparo_endometrial,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,utero_substituicao,prontuario_genitores,doacao_ovulos,doacao_embrioes,pgt,era_auxiliar,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,foliculos_antrais,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,data_inicio_agonista,utilizado_hcg,data_hcg,horario_planejado_hcg,gonapeptyl,dose_gonapeptyl,data_inicio_progesterona,progesterona_vaginal,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,motivo_nao_transferir,resultado_tratamento,analise_genetica,altura_bebe1,via_parto_bebe1,altura_bebe2,via_parto_bebe2,altura_bebe3,via_parto_bebe3,altura_bebe4,via_parto_bebe4,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,35548,156766,1,44,42,esposa,5,2025-07-02,Ciclo a Fresco FIV,Ciclo Natural,None,Hormonal,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,Não,PGT-A,None,None,None,Não,Não,Sim,Não,56.0,1.7,Não,2025-06-22,NaN,NaN,None,Insuficiência ovariana,None,None,2025-06-25,None,esposa,Não,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Ausência de embriões geneticamente normais,No transfer,Não,None,None,None,None,None,None,None,None,None,3652,10,befc3c5f586d59efb19fc31bfda9cbe8,2025-07-21 21:38:38
1,33071,156766,1,44,41,esposa,4,2025-03-23,Ciclo a Fresco FIV,Ciclo Natural,None,Hormonal,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,Não,PGT-A,None,None,None,Não,Não,Sim,None,56.0,1.7,Não,2025-03-14,NaN,NaN,None,Insuficiência ovariana,None,None,2025-03-14,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Ausência de embriões geneticamente normais,No transfer,Não,None,None,None,None,None,None,None,None,None,3719,10,dda99bb175432711ba8d4c9f47559d28,2025-07-21 21:38:38
2,29898,156766,1,43,41,esposa,3,2024-10-19,Ciclo a Fresco FIV,Convencional,None,Hormonal,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,Não,PGT-A,None,None,None,Não,Não,Sim,None,56.0,1.7,Não,2024-10-05,NaN,NaN,None,Insuficiência ovariana,None,None,2024-10-07,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Ausência de embriões geneticamente normais,No transfer,Não,None,None,None,None,None,None,None,None,None,3652,3,2caf42891ba6c61d7b7a37845e8689f9,2025-07-21 21:38:38
3,25323,156766,1,43,40,esposa,2,2024-03-01,Ciclo a Fresco FIV,Convencional,None,Hormonal,Homólogo,Criopreservado,NaN,Homólogo,Sêmen,Fresco,Homólogo,Não,NaN,Não,Não,None,Não,None,None,None,None,None,None,56.0,1.7,Não,2024-02-15,NaN,NaN,None,Insuficiência ovariana,None,None,2024-02-19,18,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,6,2024-03-07,None,None,None,Negativo,Não,None,None,None,None,None,None,None,None,None,3589,3,d97ad57d0ffd00e870896014212720ca,2025-07-21 21:38:38
4,23926,156766,1,42,40,esposa,1,2023-12-03,Ciclo a Fresco FIV,Convencional,None,Hormonal,Homólogo,Fresco,NaN,Homólogo,Sêmen,Fresco,None,Não,NaN,Não,Não,PGT-A,None,None,None,None,None,None,None,56.0,1.7,None,NaT,NaN,NaN,None,Insuficiência ovariana,None,None,2023-11-21,None,esposa,None,Não utilizado,None,None,None,None,None,None,NaT,None,None,None,No transfer,NaT,None,None,Ausência de desenvolvimento embrionário,No transfer,Não,None,None,None,None,None,None,None,None,None,3719,3,6cfdb8986a85f07c0d508e688755e2ff,2025-07-21 21:38:38


### 4. Congelamentos Embrioes


In [224]:
print("=== CONGELAMENTOS EMBRIOES DATA ===")
congelamentos_data = results['congelamentos_embrioes']
print(f"Records: {len(congelamentos_data)}")
if not congelamentos_data.empty:
    display(congelamentos_data)
else:
    print("No congelamentos embrioes data found.")


=== CONGELAMENTOS EMBRIOES DATA ===
Records: 4


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,D5,D6,Observacoes,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,29421,E1747/25,1,156766,esposa,2025-07-08,1900-01-01 10:30:00,2218/25,Não,2,2,E22891,None,None,None,Vitrificação,NaN,1.0,None,Dayane Reis,Ana Reis,Cobrar,37ec568aae4b34be676fbe0a037f3b53,2025-07-21 21:38:47
1,28342,E676/25,1,156766,esposa,2025-03-28,1900-01-01 13:40:00,873/25,Não,1,1,E22240,None,None,None,Vitrificação,1.0,NaN,None,Dayane Reis,Beatriz Aiello,Cobrar,e109b8e5b9334ffe61e585ba5fe12ccf,2025-07-21 21:38:47
2,27036,E2794/24,1,156766,esposa,2024-10-24,1900-01-01 10:45:00,3524/24,Não,1,1,E21649,None,None,None,Vitrificação,1.0,NaN,None,Bruna Lázaro Lourenço,Priscila Fabiane Dos Santos Beirigo Paternostro,Cobrar,4d4508886823389551971331ac188042,2025-07-21 21:38:47
3,24798,E574/24,1,156766,esposa,2024-03-07,1900-01-01 12:30:00,626/24,Não,1,1,E20801,DELTA,6,None,Vitrificação,NaN,1.0,None,Dayane Reis,Barbarah Silveira,None,62b44b9e7545aad81ac01c4da9b27c63,2025-07-21 21:38:47


### 5. Descongelamentos Embrioes


In [225]:
print("=== DESCONGELAMENTOS EMBRIOES DATA ===")
descongelamentos_data = results['descongelamentos_embrioes']
print(f"Records: {len(descongelamentos_data)}")
if not descongelamentos_data.empty:
    display(descongelamentos_data)
else:
    print("No descongelamentos embrioes data found.")


=== DESCONGELAMENTOS EMBRIOES DATA ===
Records: 0
No descongelamentos embrioes data found.


### 6. Embrioes Congelados


In [226]:
print("=== EMBRIOES CONGELADOS DATA ===")
embrioes_congelados_data = results['embrioes_congelados']
print(f"Records: {len(embrioes_congelados_data)}")
if not embrioes_congelados_data.empty:
    display(embrioes_congelados_data)
    # display(embrioes_congelados_data[embrioes_congelados_data['id_descongelamento']!=0])
else:
    print("No embrioes congelados data found.")


=== EMBRIOES CONGELADOS DATA ===
Records: 4


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,embriao,qualidade,comentarios,comentarios_descongelamento,ah,dia5_2,dia5_3,dia5_4,transferidos,pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
0,87258,262704,29421,0,156766,P2,E22891,#2,Blastocisto Grau 4 - B - C,None,None,None,None,None,None,Descartado,None,"XX, Monossomia chr 16, Monossomia chr 22 Feminino, Aneuploide",Sim,Aneuploide,3.7,D6,1.72,AN,4,E22891,5564d4cd77fec57eaf6ca7a77778b753,2025-08-04 19:03:04
1,83297,245946,28342,0,156766,P1,E22240,#1,Blastocisto Grau 4 - A - B,MANTER ATÉ REAVALIAÇÃO,None,None,None,None,None,None,None,"XX, Trissomia chr 22 Feminino, Aneuploide",Sim,Aneuploide,6.3,D5,N/I,C,5,E22240,c3b1541470b281ebc2dfc2bf3b832fc4,2025-07-21 21:38:57
2,78416,222558,27036,0,156766,P2,E21649,#2,Blastocisto Grau 5 - A - B,None,None,None,None,None,None,Descartado,None,"XX, Trissomia chr 13, Trissomia chr 14 Feminino, Aneuploide",Sim,Aneuploide,7.1,D5,2.45,AH,5,E21649,ccdbcdc85e8429dc670945fead9943ef,2025-07-21 21:38:57
3,70398,186310,24798,0,156766,P1,E20801,#1,Blastocisto Grau 5 - B - B,None,None,None,None,None,None,Descartado,None,Anormal/Aneuplóide Trissomia 16 Feminino,Sim,Aneuploide,6.0,D6,None,None,None,None,5f257f69f76592d152bd285482f42a23,2025-07-21 21:38:57


### 7. Clinisys Embrioes


In [227]:
print("=== CLINISYS EMBRIOES DATA ===")
clinisys_embrioes_data = results['clinisys_embrioes']
print(f"Records: {len(clinisys_embrioes_data)}")
if not clinisys_embrioes_data.empty:
    display(clinisys_embrioes_data.head(10))
else:
    print("No clinisys embrioes data found.")


=== CLINISYS EMBRIOES DATA ===
Records: 19


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [228]:
clinisys_embrioes_data[clinisys_embrioes_data['oocito_id'].isin([184181])]

,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [229]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento'
]

clinisys_embrioes_data.loc[clinisys_embrioes_data['oocito_embryo_number'].isin([33,35]), colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento


### 8. Embryoscope Embrioes


In [230]:
print("=== EMBRYOSCOPE EMBRIOES DATA ===")
embryoscope_embrioes_data = results['embryoscope_embrioes']
print(f"Records: {len(embryoscope_embrioes_data)}")
if not embryoscope_embrioes_data.empty:
    display(embryoscope_embrioes_data)
else:
    print("No embryoscope embrioes data found.")


=== EMBRYOSCOPE EMBRIOES DATA ===
Records: 15


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,prontuario,patient_unit_huntington,treatment_PatientIDx,treatment_TreatmentName,treatment_unit_huntington,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Value_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Value_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Value_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_PatientIDx,embryo_TreatmentName
0,PC1P7BHG_45263.5119626852,156766,"HEINIGER, NICOLE I.",12/02/1981,1981-02-12,156766,Ibirapuera,PC1P7BHG_45263.5119626852,2025-1357,Ibirapuera,D2025.07.02_S04183_I3027_P-2,2025-07-08,3.7,ADMIN,KIDScoreD5 v3.3,"Complex aneuploid -16, -22, XX",AE2,Freeze,None,2025-07-02 15:40:00,3027,11,2,2,Ibirapuera,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,N

In [231]:
embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')

C:\Users\FilipeFurlanBellotti\AppData\Local\Temp\ipykernel_40916\2666312698.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')


,patient_PatientIDx,patient_PatientID,patient_FirstName,patient_LastName,patient_DateOfBirth,prontuario,patient_unit_huntington,treatment_PatientIDx,treatment_TreatmentName,treatment_unit_huntington,embryo_EmbryoID,embryo_KIDDate,embryo_KIDScore,embryo_KIDUser,embryo_KIDVersion,embryo_Description,embryo_EmbryoDescriptionID,embryo_EmbryoFate,embryo_FertilizationMethod,embryo_FertilizationTime,embryo_InstrumentNumber,embryo_Position,embryo_WellNumber,embryo_embryo_number,embryo_unit_huntington,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Value_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Timestamp_EVEN2,embryo_Value_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Timestamp_EVEN4,embryo_Value_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Timestamp_EVEN8,embryo_Value_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Timestamp_FRAG2,embryo_Value_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Timestamp_FRAG4,embryo_Value_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Timestamp_FRAG8,embryo_Value_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Value_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Timestamp_ICM,embryo_Value_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Timestamp_Line,embryo_Value_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Timestamp_MN2Type,embryo_Value_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Value_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Value_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Timestamp_PN,embryo_Value_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Timestamp_Pulsing,embryo_Value_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Timestamp_Strings,embryo_Value_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Timestamp_TE,embryo_Value_TE,embryo_Name_t2,embryo_Time_t2,embryo_Timestamp_t2,embryo_Value_t2,embryo_Name_t3,embryo_Time_t3,embryo_Timestamp_t3,embryo_Value_t3,embryo_Name_t4,embryo_Time_t4,embryo_Timestamp_t4,embryo_Value_t4,embryo_Name_t5,embryo_Time_t5,embryo_Timestamp_t5,embryo_Value_t5,embryo_Name_t6,embryo_Time_t6,embryo_Timestamp_t6,embryo_Value_t6,embryo_Name_t7,embryo_Time_t7,embryo_Timestamp_t7,embryo_Value_t7,embryo_Name_t8,embryo_Time_t8,embryo_Timestamp_t8,embryo_Value_t8,embryo_Name_t9,embryo_Time_t9,embryo_Timestamp_t9,embryo_Value_t9,embryo_Name_tB,embryo_Time_tB,embryo_Timestamp_tB,embryo_Value_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Timestamp_tEB,embryo_Value_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Timestamp_tHB,embryo_Value_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Timestamp_tM,embryo_Value_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Timestamp_tPB2,embryo_Value_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Timestamp_tPNa,embryo_Value_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Timestamp_tPNf,embryo_Value_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Timestamp_tSB,embryo_Value_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Timestamp_tSC,embryo_Value_tSC,embryo_PatientIDx,embryo_TreatmentName


### 9. Embryoscope Clinisys Combined


In [232]:
print("=== EMBRYOSCOPE CLINISYS COMBINED DATA ===")
combined_data = results['embryoscope_clinisys_combined']
print(f"Records: {len(combined_data)}")
if not combined_data.empty:
    display(combined_data.head(10))
else:
    print("No embryoscope clinisys combined data found.")


=== EMBRYOSCOPE CLINISYS COMBINED DATA ===
Records: 19


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D4,oocito_NCelulas_D4,oocito_Compactando_D4,oocito_MassaInterna_D4,oocito_Trofoblasto_D4,oocito_NClivou_D5,oocito_NCelulas_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_NCelulas_D6,oocito_Compactando_D6,oocito_MassaInterna_D6,oocito_Trofoblasto_D6,oocito_NClivou_D7,oocito_NCelulas_D7,oocito_Compactando_D7,oocito_MassaInterna_D7,oocito_Trofoblasto_D7,oocito_score_maia,oocito_relatorio_ia,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,cong_em_id,cong_em_CodCongelamento,cong_em_Unidade,cong_em_prontuario,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,cong_em_Observacoes,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,descong_em_id,descong_em_CodDescongelamento,descong_em_Unidade,descong_em_prontuario,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,descong_em_Ciclo,descong_em_Identificacao,descong_em_CodCongelamento,descong_em_Tambor,descong_em_Cane,descong_em_PailletesDescongeladas,descong_em_Tecnica,descong_em_DataTransferencia,descong_em_Incubadora,descong_em_transferido

In [233]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'micro_codigo_ficha', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'treatment_TreatmentName', 'embryo_EmbryoID', 'embryo_KIDScore', 'embryo_Description', 'embryo_EmbryoDescriptionID', 'embryo_embryo_number',
    'emb_cong_transferidos', 'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento', 'trat_tentativa'
]

# combined_data.loc[combined_data['oocito_id_micromanipulacao'].isin([21961]), colunas]
# combined_data.loc[combined_data['embryo_EmbryoDescriptionID'].isin(['AA5']), colunas] 
combined_data.loc[:, colunas]

,oocito_id,oocito_id_micromanipulacao,oocito_embryo_number,oocito_TCD,micro_codigo_ficha,micro_numero_caso,micro_Data_DL,micro_data_procedimento,cong_em_id,cong_em_CodCongelamento,cong_em_Ciclo,cong_em_NEmbrioes,treatment_TreatmentName,embryo_EmbryoID,embryo_KIDScore,embryo_Description,embryo_EmbryoDescriptionID,embryo_embryo_number,emb_cong_transferidos,emb_cong_id,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,emb_cong_embriao,descong_em_id,descong_em_transferidos_transferencia,descong_em_CodDescongelamento,descong_em_Ciclo,descong_em_DataDescongelamento,descong_em_DataTransferencia,trat_id,trat_data_procedimento,trat_data_transferencia,trat_resultado_tratamento,trat_tentativa
0,262703,26281,1,Descartado,26281,2218/25,2025-07-02,2025-07-02,<NA>,None,None,<NA>,2025-1357,D2025.07.02_S04183_I3027_P-1,1.6,None,AE1,1,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,35548,2025-07-02,NaT,No transfer,5
1,262704,26281,2,Criopreservado,26281,2218/25,2025-07-02,2025-07-02,29421,E1747/25,2218/25,2,2025-1357,D2025.07.02_S04183_I3027_P-2,3.7,"Complex aneuploid -16, -22, XX",AE2,2,Descartado,87258,262704,29421,0,#2,<NA>,None,None,None,NaT,NaT,35548,2025-07-02,NaT,No transfer,5
2,262705,26281,3,Descartado,26281,2218/25,2025-07-02,2025-07-02,<NA>,None,None,<NA>,None,None,None,None,None,<NA>,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,35548,2025-07-02,NaT,No transfer,5
3,245946,24791,1,Criopreservado,24791,873/25,2025-03-23,2025-03-23,28342,E676/25,873/25,1,2025-537,D2025.03.23_S04114_I3166_P-1,6.3,"Aneuploid +22, XX",AD1,1,None,83297,245946,28342,0,#1,<NA>,None,None,None,NaT,NaT,33071,2025-03-23,NaT,No transfer,4
4,222557,22794,1,Descartado,22794,3524/24,2024-10-19,2024-10-19,<NA>,None,None,<NA>,2024-1700,D2024.10.19_S03853_I3166_P-1,NA,None,AC1,1,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,29898,2024-10-19,NaT,No transfer,3
5,222558,22794,2,Criopreservado,22794,3524/24,2024-10-19,2024-10-19,27036,E2794/24,3524/24,1,2024-1700,D2024.10.19_S03853_I3166_P-2,7.1,"Complex aneuploid +13, +14, XX",AC2,2,Descartado,78416,222558,27036,0,#2,<NA>,None,None,None,NaT,NaT,29898,2024-10-19,NaT,No transfer,3
6,222559,22794,3,Descartado,22794,3524/24,2024-10-19,2024-10-19,<NA>,None,None,<NA>,2024-1700,D2024.10.19_S03853_I3166_P-3,1.7,None,AC3,3,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,29898,2024-10-19,NaT,No transfer,3
7,186310,19589,1,Criopreservado,19589,626/24,2024-03-01,2024-03-01,24798,E574/24,626/24,1,2024-311,D2024.03.01_S03461_I3166_P-1,6,"Aneuploid +16, XX",AB1,1,Descartado,70398,186310,24798,0,#1,<NA>,None,None,None,NaT,NaT,25323,2024-03-01,2024-03-07,Negativo,2
8,186311,19589,2,Descartado,19589,626/24,2024-03-01,2024-03-01,<NA>,None,None,<NA>,2024-311,D2024.03.01_S03461_I3166_P-2,NA,None,AB2,2,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,25323,2024-03-01,2024-03-07,Negativo,2
9,186312,19589,3,Descartado,19589,626/24,2024-03-01,2024-03-01,<NA>,None,None,<NA>,2024-311,D2024.03.01_S03461_I3166_P-3,NA,None,AB3,3,None,<NA>,<NA>,<NA>,<NA>,None,<NA>,None,None,None,NaT,NaT,25323,2024-03-01,2024-03-07,Negativo,2


### 11. Planilha Embriologia


In [234]:
print("=== PLANILHA EMBRIOLOGIA DATA ===")
planilha_data = results['planilha_embriologia']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA ===
Records: 0
No planilha embriologia data found.


### 12. Congelamentos Ovulos


In [235]:
print("=== CONGELAMENTOS OVULOS DATA ===")
congelamentos_ovulos_data = results['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No congelamentos ovulos data found.")


=== CONGELAMENTOS OVULOS DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,NOvulos,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Motivo,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,9492,EGG2235,1,156766,esposa,2018-04-19,1900-01-01 10:00:00,-,8,2,EGG2235,DELTA,2,None,Vitrificação,Preservação da Fertilidade,Ana Reis,Mariana Nicolielo,None,64715224e53c2d37c186799cd09829e4,2025-07-21 21:38:49


### 13. Descongelamentos Ovulos


In [236]:
print("=== DESCONGELAMENTOS OVULOS DATA ===")
descongelamentos_ovulos_data = results['descongelamentos_ovulos']
print(f"Records: {len(descongelamentos_ovulos_data)}")
if not descongelamentos_ovulos_data.empty:
    display(descongelamentos_ovulos_data)
else:
    print("No descongelamentos ovulos data found.")


=== DESCONGELAMENTOS OVULOS DATA ===
Records: 1


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,3702,DO133/24,1,156766,0,2018-04-19,2024-03-01,-,EGG2235,EGG2235,DELTA,2,2,Desvitrificação,None,Beatriz Aiello,Priscila Fabiane Dos Santos Beirigo Paternostro,ce52e341b748a39d08e90ac443066323,2025-07-21 21:38:53


### 14. Ovulos Congelados


In [237]:
print("=== OVULOS CONGELADOS DATA ===")
ovulos_congelados_data = results['ovulos_congelados']
print(f"Records: {len(ovulos_congelados_data)}")
if not ovulos_congelados_data.empty:
    display(ovulos_congelados_data)
else:
    print("No ovulos congelados data found.")


=== OVULOS CONGELADOS DATA ===
Records: 8


,id,id_congelamento,id_descongelamento,prontuario,pailletes,ovulo,comentarios,dia2,maturidade,dia2_2,tanque_amostra,caneca_amostra,rack_amostra,hash,extraction_timestamp
0,56751,9492,3702,156766,2,8.0,None,MII,MII,None,None,None,None,c6d4550b168431aa5520afa2a5e8e511,2025-10-14 21:04:28
1,56750,9492,3702,156766,2,7.0,None,MII,MII,None,None,None,None,f8ebe837d94f55284cc85e79821e949c,2025-10-14 21:04:28
2,56749,9492,3702,156766,2,6.0,None,MII,MII,None,None,None,None,ba05b0d6051f760a7b3d0075cd5cc4b8,2025-10-14 21:04:28
3,56748,9492,3702,156766,2,5.0,None,MII,MII,None,None,None,None,c766f17dae602f0b92519f11b982b3ac,2025-10-14 21:04:28
4,56747,9492,3702,156766,1,4.0,None,MII,MII,None,None,None,None,996aa4277528a7dc092619488054cfcc,2025-10-14 21:04:28
5,56746,9492,3702,156766,1,3.0,None,MII,MII,None,None,None,None,2849e920956a328ba428cc1fa122bf65,2025-10-14 21:04:28
6,56745,9492,3702,156766,1,2.0,None,MII,MII,None,None,None,None,743ebc9ef744cb7ace41589cf2d966f6,2025-10-14 21:04:28
7,56744,9492,3702,156766,1,1.0,None,MII,MII,None,None,None,None,0b45242b8fae9058d76d8f32889c872c,2025-10-14 21:04:28


### 12. Planilha Embryoscope Combined


In [238]:
print("=== PLANILHA EMBRYOSCOPE COMBINED DATA ===")
combined_planilha_data = results['planilha_embryoscope_combined']
print(f"Records: {len(combined_planilha_data)}")
if not combined_planilha_data.empty:
    display(combined_planilha_data)
else:
    print("No planilha embryoscope combined data found.")


=== PLANILHA EMBRYOSCOPE COMBINED DATA ===


Records: 19


,oocito_id,oocito_id_micromanipulacao,oocito_diaseguinte,oocito_Maturidade,oocito_RC,oocito_Embriologista,oocito_PI,oocito_TCD,oocito_PGD,oocito_ResultadoPGD,oocito_IdentificacaoPGD,oocito_Fertilizacao,oocito_CorpusculoPolar,oocito_ComentariosDepois,oocito_OocitoDoado,oocito_OrigemOocito,oocito_InseminacaoOocito,oocito_NClivou_D3,oocito_NCelulas_D3,oocito_Frag_D3,oocito_Blastomero_D3,oocito_NClivou_D5,oocito_Compactando_D5,oocito_MassaInterna_D5,oocito_Trofoblasto_D5,oocito_NClivou_D6,oocito_Compactando_D6,oocito_flag_embryoscope,oocito_embryo_number,micro_codigo_ficha,micro_numero_caso,micro_prontuario,micro_IdadeEsposa_DG,micro_IdadeMarido_DG,micro_Data_DL,micro_horario_inicial_fert,micro_horario_final_fert,micro_Aspiracao_DL,micro_medico_id,micro_responsavel_puncao_id,micro_recepcao_ovulos,micro_Denud_DL,micro_checagem_fertilizacao,micro_checagem_d3,micro_checagem_d5,micro_checagem_d6,micro_checagem_d7,micro_foliculos,micro_oocitos,micro_ICSIDescongelados,micro_pgd_oocito,micro_pgd_d5,micro_data_procedimento,micro_tecnica_injetados,micro_motilidade,micro_morfologia_injetados,micro_gradiente_descontinuo,micro_unidade_id,micro_responsavel_denud_id,micro_profissional_fertilizacao_id,micro_profissional_d3_id,micro_profissional_d5_id,micro_profissional_d6_id,micro_profissional_d7_id,micro_materiais_insumos,micro_local_coleta,micro_viscosidade,micro_volume_amostra_utilizada,micro_morfologia_pre,micro_morfologia_pos,micro_zymot,micro_observacoes_resumo_ciclos,micro_agulha_icsi,micro_lote_agulha_icsi,micro_validade_agulha_icsi,micro_agulha_aspir,micro_lote_agulha_aspir,micro_validade_agulha_aspir,micro_incubadora,micro_abstinencia_dias,micro_meios,micro_responsavel_oocitos_id,micro_fonte,micro_inicio_ps,micro_termino_ps,micro_volume_ps1,micro_CongelamentoOvulos,micro_concentracao_progressivos_pre,micro_conc_ps1,micro_EstadoSptz,micro_check_gradiente_ps,micro_check_swimup,micro_volume_ps2,micro_concentracao_progressivos_pos,micro_conc_ps2,micro_comentarios_ps,micro_responsavel_ps,micro_timelapse,micro_transferidos,micro_Transfer_D5,micro_medico_responsavel_id,micro_azoospermia,micro_Observacoes,micro_responsavel_labfiv,micro_responsavel_labfiv_data,micro_tipo_paciente,micro_alteracoes_oocitarias_normal,micro_alteracoes_oocitarias_granulacao,micro_alteracoes_oocitarias_pelucida,micro_alteracoes_oocitarias_oval,micro_alteracoes_oocitarias_inclusoes,micro_alteracoes_oocitarias_pervitelineo_aumentado,micro_alteracoes_oocitarias_irregular,micro_alteracoes_oocitarias_necrotica,micro_alteracoes_oocitarias_pervitelineo_granular,micro_alteracoes_oocitarias_ser,micro_alteracoes_oocitarias_corpusculo_frag,micro_alteracoes_oocitarias_vacuolos,micro_alteracoes_oocitarias_corpusculo_aum,micro_alteracoes_oocitarias_membrana,micro_alteracoes_oocitarias_corpusculo_peq,micro_alteracoes_oocitarias_citoplasma,micro_alteracoes_oocitarias_corpusculo_deg,micro_controle_anual,micro_maia,id_1,cong_em_CodCongelamento,cong_em_Unidade,prontuario_1,cong_em_paciente,cong_em_Data,cong_em_Hora,cong_em_Ciclo,cong_em_CicloRecongelamento,cong_em_NEmbrioes,cong_em_NPailletes,cong_em_Identificacao,cong_em_Tambor,cong_em_Cane,cong_em_Cane2,cong_em_Tecnica,cong_em_D5,cong_em_D6,Observacoes_1,cong_em_BiologoFIV,cong_em_BiologoFIV2,cong_em_status_financeiro,id_2,descong_em_CodDescongelamento,Unidade_1,prontuario_2,descong_em_doadora,descong_em_DataCongelamento,descong_em_DataDescongelamento,Ciclo_1,Identificacao_1,CodCongelamento_1,Tambor_1,Cane_1,descong_em_PailletesDescongeladas,Tecnica_1,descong_em_DataTransferencia,Incubadora_1,descong_em_transferidos_transferencia,descong_em_cateter_transferencia,descong_em_lote_transferencia,descong_em_validade_transferencia,descong_em_sangue_interno_transferencia,descong_em_sangue_externo_transferencia,descong_em_retorno_transferencia,descong_em_responsavel_transferencia,Observacoes_2,BiologoFIV_1,BiologoFIV2_1,id_3,emb_cong_id_oocito,emb_cong_id_congelamento,emb_cong_id_descongelamento,prontuario_3,emb_cong_pailletes,emb_cong_paill